# Ag-Cu Binary Phase Diagram with OpenCALPHAD.jl

This notebook demonstrates how to calculate and plot a binary phase diagram
using OpenCALPHAD.jl. We will compute the FCC miscibility gap in the Ag-Cu system.

## References
- TDB data from COST 531 database
- OpenCALPHAD.jl: https://github.com/your-username/OpenCALPHAD.jl

## 1. Setup

First, load the required packages.

In [ ]:
using OpenCALPHAD
using Plots
gr()  # Use GR backend for plotting

## 2. Load Thermodynamic Database

Load the Ag-Cu TDB file and extract the FCC phase.

In [ ]:
# Load TDB file
tdb_path = joinpath(@__DIR__, "..", "reftest", "tdb", "agcu.TDB")
db = read_tdb(tdb_path)

# Get FCC phase
fcc = get_phase(db, "FCC_A1")

println("Database loaded:")
println("  Elements: ", length(db.elements))
println("  Phases: ", length(db.phases))
println("  FCC sublattices: ", fcc.sites)

## 3. Gibbs Energy Curve at Fixed Temperature

Calculate and plot the Gibbs energy as a function of composition at T = 1000 K.
The common tangent construction shows the equilibrium compositions.

In [ ]:
# Temperature
T = 1000.0  # K

# Scan composition
solver = GridSearchSolver(n_points=101)
scan = scan_composition(fcc, T, db, solver)

# Plot G-x curve
p1 = plot(scan.x_grid, scan.G_values ./ 1000,
    xlabel="x(Ag)",
    ylabel="Gibbs Energy [kJ/mol]",
    title="Ag-Cu FCC Gibbs Energy at T=$(Int(T)) K",
    linewidth=2,
    label="G(x)",
    legend=:topright,
    size=(700, 500)
)

# Find miscibility gap and add common tangent
gap = find_miscibility_gap(fcc, T, db)
if !isnothing(gap)
    scatter!(p1, [gap.x1, gap.x2], [gap.G1, gap.G2] ./ 1000,
        markersize=8, color=:red, label="Equilibrium")
    plot!(p1, [gap.x1, gap.x2], [gap.G1, gap.G2] ./ 1000,
        linestyle=:dash, color=:red, linewidth=2, label="Common tangent")
    println("Miscibility gap at T=$(T) K:")
    println("  Cu-rich: x(Ag) = $(round(gap.x1, digits=4))")
    println("  Ag-rich: x(Ag) = $(round(gap.x2, digits=4))")
end

p1

## 4. Binary Phase Diagram (T-x)

Calculate the phase diagram by sweeping temperature from 600 K to 1200 K.
The miscibility gap boundaries are computed at each temperature.

In [ ]:
# Calculate phase diagram
T_start = 600.0   # K
T_end = 1200.0    # K
T_step = 25.0     # K

println("Calculating phase diagram...")
result = map_phase_diagram(fcc, db, T_start, T_end, T_step)

# Count converged points
n_converged = count(p -> p.converged, result.points)
println("Converged: $(n_converged) / $(length(result.points)) points")

In [ ]:
# Extract data for plotting
converged = filter(p -> p.converged, result.points)
T_vals = [p.temperature for p in converged]
x1_vals = [p.compositions[1] for p in converged]  # Cu-rich boundary
x2_vals = [p.compositions[2] for p in converged]  # Ag-rich boundary

# Plot phase diagram
p2 = plot(x1_vals, T_vals,
    label="Cu-rich boundary",
    xlabel="x(Ag)",
    ylabel="Temperature [K]",
    title="Ag-Cu Binary Phase Diagram (FCC Miscibility Gap)",
    linewidth=2,
    legend=:top,
    size=(700, 500)
)
plot!(p2, x2_vals, T_vals, label="Ag-rich boundary", linewidth=2)

# Fill two-phase region
x_fill = vcat(x1_vals, reverse(x2_vals))
T_fill = vcat(T_vals, reverse(T_vals))
plot!(p2, x_fill, T_fill,
    fillrange=minimum(T_vals),
    fillalpha=0.2,
    label="Two-phase (α + α')",
    linewidth=0
)

# Add annotations
annotate!(p2, 0.15, 900, text("α (Cu-rich)", 10))
annotate!(p2, 0.85, 900, text("α' (Ag-rich)", 10))
annotate!(p2, 0.5, 750, text("α + α'", 10))

p2

## 5. Using Julia DSL (Alternative Method)

Instead of reading from a TDB file, we can define the thermodynamic model
directly using Julia functions. This is useful for custom models or education.

In [ ]:
# Define GHSER functions as Julia functions
ghser_ag(T) = -7209.512 + 118.202013*T - 23.8463314*T*log(T) - 
              0.001790585*T^2 - 3.98587e-7*T^3 - 12011.0/T

ghser_cu(T) = -7770.458 + 130.485235*T - 24.112392*T*log(T) - 
              0.00265684*T^2 + 1.29223e-7*T^3 + 52478.0/T

# Check AD compatibility
println("AD compatibility check:")
println("  ghser_ag: ", validate_ad_compatibility(ghser_ag))
println("  ghser_cu: ", validate_ad_compatibility(ghser_cu))

In [ ]:
# Create database using DSL
db_dsl = Database()
add_elements!(db_dsl, [:AG, :CU])

# Create FCC phase with Julia functions
fcc_dsl = Phase("FCC_A1", [1.0], [[:AG, :CU]])

# Set endmember Gibbs energies
set_G!(fcc_dsl, [:AG], ghser_ag)
set_G!(fcc_dsl, [:CU], ghser_cu)

# Set interaction parameters (Redlich-Kister)
set_L!(fcc_dsl, [:AG, :CU], 0, T -> 35620.29 - 1.84985*T)
set_L!(fcc_dsl, [:AG, :CU], 1, T -> -4890.74 - 0.66754*T)

add_phase!(db_dsl, fcc_dsl)

println("DSL Database created:")
println("  Elements: ", length(db_dsl.elements))
println("  Julia parameters: ", length(fcc_dsl.julia_parameters))

In [ ]:
# Compare TDB vs DSL at T=1000K
T = 1000.0
println("\nGibbs energy comparison (TDB vs DSL) at T=$(T) K:")
println("-"^50)

for x_ag in [0.1, 0.3, 0.5, 0.7, 0.9]
    # DSL calculation
    y = [x_ag 1.0-x_ag]
    G_dsl = calculate_gibbs_energy(fcc_dsl, T, y)
    println("x(Ag)=$(x_ag): G = $(round(G_dsl, digits=1)) J/mol")
end

## 6. Summary

This notebook demonstrated:

1. **Loading TDB files** - Standard CALPHAD database format
2. **Gibbs energy curves** - G(x) with common tangent construction  
3. **Phase diagram calculation** - T-x diagram with miscibility gap
4. **Julia DSL** - Define thermodynamic models using Julia functions

### Key Features of OpenCALPHAD.jl

- **Pure Julia** - No external binary dependencies
- **Automatic differentiation** - ForwardDiff.jl for accurate gradients
- **Julia DSL** - Custom models as regular Julia functions
- **Plots.jl integration** - Easy visualization